In [ ]:
import pandas as pd
import os
import tqdm
import plotly.express as px
import numpy as np
os.chdir(r'D:\Projects\football-odds-analysis')
from utils.preprocess import preprocess_pipeline, preprocess_odds_results
from utils.favorite_longshot import get_favorite_bets, get_longshot_bets
from utils.bet_payout_engine import get_bet_results
from input_variables import CO_CODES

PORTFOLIO_STARTING_CASH = 1000
START_DATE = '2005-01-01'
FAVORITE_PROBABILITY_THRESHOLD = 0.98
LONGSHOT_PROBABILITY_THRESHOLD = 0.1
MAX_EXPOSURE_PER_DAY = 0.05

odds_results_data = preprocess_odds_results('all_avail_games.csv')
bookmakers_data = preprocess_pipeline('all_avail_games.csv')

bookmakers_odds = odds_results_data['Odds']
game_results = odds_results_data['Results']

favorite_bets = get_favorite_bets(bookmakers_odds, FAVORITE_PROBABILITY_THRESHOLD)
longshot_bets = get_longshot_bets(bookmakers_odds, LONGSHOT_PROBABILITY_THRESHOLD)

## Favorite Bets

In [ ]:
all_games_dates = game_results['Date'][game_results['Date'] > START_DATE].sort_values().unique()
favorite_portfolio_value = {all_games_dates[0]: PORTFOLIO_STARTING_CASH}

for i, date in tqdm.tqdm(enumerate(all_games_dates)):
    if i == 0:
        continue
    todays_portfolio_value = favorite_portfolio_value[all_games_dates[i-1]]
    dates_games = game_results[game_results['Date'] == date]
    dates_games_signals = dates_games[dates_games.index.isin(list(favorite_bets.keys()))]
    
    if len(dates_games_signals) > 0:
        per_set_cash = (todays_portfolio_value * MAX_EXPOSURE_PER_DAY)/len(dates_games_signals)
        game_payout = todays_portfolio_value - (per_set_cash)*len(dates_games_signals)
        for game in dates_games_signals.index:
            bet = favorite_bets[game]
            company = bet[:-1]
            bet_side = bet[-1]
            bet_payoff = get_bet_results(bookmakers_data, game, bet_side, CO_CODES[company], per_set_cash)
            game_payout = game_payout + bet_payoff
        
        favorite_portfolio_value[date] = game_payout
        continue
    favorite_portfolio_value[date] = todays_portfolio_value

portfolio = pd.DataFrame.from_dict([favorite_portfolio_value]).T
portfolio = portfolio.sort_index()
portfolio.columns = ['Betting_Portfolio_Value']

In [ ]:
fig = px.line(x = portfolio.index ,
              y = portfolio['Betting_Portfolio_Value'],
              title = f'Cumulative Value of Favorite Betting Strategy Starting with {str(int(PORTFOLIO_STARTING_CASH))}$')
fig.show()

## Longshot Bets

In [ ]:
all_games_dates = game_results['Date'][game_results['Date'] > START_DATE].sort_values().unique()
longshot_portfolio_value = {all_games_dates[0]: PORTFOLIO_STARTING_CASH}

for i, date in tqdm.tqdm(enumerate(all_games_dates)):
    if i == 0:
        continue
    todays_portfolio_value = longshot_portfolio_value[all_games_dates[i-1]]
    dates_games = game_results[game_results['Date'] == date]
    dates_games_signals = dates_games[dates_games.index.isin(list(longshot_bets.keys()))]
    
    if len(dates_games_signals) > 0:
        per_set_cash = (todays_portfolio_value * MAX_EXPOSURE_PER_DAY)/len(dates_games_signals)
        game_payout = todays_portfolio_value - (per_set_cash)*len(dates_games_signals)
        for game in dates_games_signals.index:
            bet = longshot_bets[game]
            company = bet[:-1]
            bet_side = bet[-1]
            bet_payoff = get_bet_results(bookmakers_data, game, bet_side, CO_CODES[company], per_set_cash)
            game_payout = game_payout + bet_payoff
        
        longshot_portfolio_value[date] = game_payout
        continue
    longshot_portfolio_value[date] = todays_portfolio_value

portfolio = pd.DataFrame.from_dict([longshot_portfolio_value]).T
portfolio = portfolio.sort_index()
portfolio.columns = ['Betting_Portfolio_Value']

In [ ]:
fig = px.line(x = portfolio.index ,
              y = portfolio['Betting_Portfolio_Value'],
              title = f'Cumulative Value of Longshot Betting Strategy Starting with {str(int(PORTFOLIO_STARTING_CASH))}$')
fig.show()